In [286]:
import pandas as pd 

columns = ['user_id', 'page', 'date', 'platform', 'gender']
data = pd.read_csv('click_stream3.csv', names=columns, parse_dates=[2])

In [287]:
data['page'].unique()

array(['1_home_page', '2_search_page', '3_payment_page',
       '4_payment_confirmation_page'], dtype=object)

In [288]:
funnel = data[['user_id', 'page']].groupby('page').count()
funnel

,user_id
page,
1_home_page,90400
2_search_page,45200
3_payment_page,6030
4_payment_confirmation_page,452


In [289]:
funnel.columns

Index(['user_id'], dtype='object')

In [290]:
data['date'].apply(lambda x: x.month).unique()

array([2, 3, 1, 4])

In [291]:
data['month'] = data['date'].map(lambda x: x.month)
data.head()

,user_id,page,date,platform,gender,month
0,313593,1_home_page,2015-02-26,Desktop,Female,2
1,468315,1_home_page,2015-02-21,Desktop,Male,2
2,264005,1_home_page,2015-03-25,Desktop,Female,3
3,290784,1_home_page,2015-03-14,Desktop,Male,3
4,639104,1_home_page,2015-01-03,Desktop,Female,1


In [292]:
funnel_dynamic = data.pivot_table(
    index='page', columns='month', 
    values='user_id', aggfunc='count'
)

funnel_dynamic

month,1,2,3,4
page,,,,
1_home_page,22600,22600,22600,22600
2_search_page,13554,13687,8879,9080
3_payment_page,2390,2412,631,597
4_payment_confirmation_page,189,173,44,46


In [293]:
funnel_dynamic.div(funnel_dynamic.max(), level=0)

month,1,2,3,4
page,,,,
1_home_page,1.000000,1.000000,1.000000,1.000000
2_search_page,0.599735,0.605619,0.392876,0.401770
3_payment_page,0.105752,0.106726,0.027920,0.026416
4_payment_confirmation_page,0.008363,0.007655,0.001947,0.002035


In [294]:
data.pivot_table(index='platform', values='user_id', aggfunc='count')

,user_id
platform,
Desktop,93460
Mobile,48622


In [295]:
funnel_platform_dynamic = (
    data[['platform', 'month', 'page', 'user_id']]
    .groupby(['platform', 'month', 'page'])
    .count()
    .rename({'user_id': 'visits'}, axis=1)
).pivot_table(index='page', columns=['platform', 'month'], values='visits')

funnel_platform_dynamic

platform                    Desktop                      Mobile              \
month                             1      2      3      4      1     2     3   
page                                                                          
1_home_page                   15050  15050  15050  15050   7550  7550  7550   
2_search_page                  7529   7632   7370   7569   6025  6055  1509   
3_payment_page                 1221   1191    296    302   1169  1221   335   
4_payment_confirmation_page      60     54     14     22    129   119    30   

platform                           
month                           4  
page                               
1_home_page                  7550  
2_search_page                1511  
3_payment_page                295  
4_payment_confirmation_page    24

In [296]:
funnel_platform_dynamic.div(funnel_platform_dynamic.max(), level=1) * 100

platform                        Desktop                                      \
month                                 1           2           3           4   
page                                                                          
1_home_page                  100.000000  100.000000  100.000000  100.000000   
2_search_page                 50.026578   50.710963   48.970100   50.292359   
3_payment_page                 8.112957    7.913621    1.966777    2.006645   
4_payment_confirmation_page    0.398671    0.358804    0.093023    0.146179   

platform                         Mobile                                      
month                                 1           2           3           4  
page                                                                         
1_home_page                  100.000000  100.000000  100.000000  100.000000  
2_search_page                 79.801325   80.198675   19.986755   20.013245  
3_payment_page                15.483444   16.172185    4.437086    3.907285  
4_payment_confirmation_page    1.708609    1.576159    0.397351    0.317881

#### Посмотрим на есть ли перекос в аудитории по половому признаку.

In [297]:
data[['gender', 'user_id']].groupby('gender').count() / data.user_id.count()

,user_id
gender,
Female,0.500359
Male,0.499641


Перекоса по половому признаку не наблюдается. Посмотрим, в динамике.

In [298]:
data.pivot_table(index='gender', columns='month', values='user_id', aggfunc='count')

month,1,2,3,4
gender,,,,
Female,19679,19340,16067,16006
Male,19054,19532,16087,16317


Тоже ничего интересного. Тогда можно посмотреть на предпочтение устройств полами.

In [299]:
gender_platform = data.pivot_table(
    index='gender', columns=['month', 'platform'], 
    values='user_id', aggfunc='count'
)
gender_platform

month          1              2              3              4       
platform Desktop Mobile Desktop Mobile Desktop Mobile Desktop Mobile
gender                                                              
Female     12055   7624   11879   7461   11387   4680   11371   4635
Male       11805   7249   12048   7484   11343   4744   11572   4745

In [300]:
gender = data.pivot_table(index='gender', columns='month', values='user_id', aggfunc='count')
gender_platform.div(gender, level=0)

month            1                   2                   3            \
platform   Desktop    Mobile   Desktop    Mobile   Desktop    Mobile   
gender                                                                 
Female    0.612582  0.387418  0.614219  0.385781  0.708720  0.291280   
Male      0.619555  0.380445  0.616834  0.383166  0.705103  0.294897   

month            4            
platform   Desktop    Mobile  
gender                        
Female    0.710421  0.289579  
Male      0.709199  0.290801

И снова паритет. Посмотрим, как пол связан с конверсией.

In [301]:
final_funnel = (
    data.pivot_table(
        index='page', columns=['month', 'gender'], 
        values='user_id', aggfunc='count'
    )
)
final_funnel

month                            1             2             3             4  \
gender                      Female   Male Female   Male Female   Male Female   
page                                                                           
1_home_page                  11414  11186  11199  11401  11250  11350  11212   
2_search_page                 6913   6641   6820   6867   4454   4425   4489   
3_payment_page                1243   1147   1235   1177    337    294    285   
4_payment_confirmation_page    109     80     86     87     26     18     20   

month                               
gender                        Male  
page                                
1_home_page                  11388  
2_search_page                 4591  
3_payment_page                 312  
4_payment_confirmation_page     26

In [302]:
final_funnel.div(funnel_dynamic, level=0)

month                               1                   2                   3  \
gender                         Female      Male    Female      Male    Female   
page                                                                            
1_home_page                  0.505044  0.494956  0.495531  0.504469  0.497788   
2_search_page                0.510034  0.489966  0.498283  0.501717  0.501633   
3_payment_page               0.520084  0.479916  0.512023  0.487977  0.534073   
4_payment_confirmation_page  0.576720  0.423280  0.497110  0.502890  0.590909   

month                                         4            
gender                           Male    Female      Male  
page                                                       
1_home_page                  0.502212  0.496106  0.503894  
2_search_page                0.498367  0.494383  0.505617  
3_payment_page               0.465927  0.477387  0.522613  
4_payment_confirmation_page  0.409091  0.434783  0.565217

Кажется, что столбец 'gender' просто равномерно размазан по всем пользователям и никакой дополнительной информации не привносит.

In [303]:
final_funnel.div(final_funnel.max(), level=0)

month                               1                   2                   3  \
gender                         Female      Male    Female      Male    Female   
page                                                                            
1_home_page                  1.000000  1.000000  1.000000  1.000000  1.000000   
2_search_page                0.605660  0.593689  0.608983  0.602316  0.395911   
3_payment_page               0.108901  0.102539  0.110278  0.103237  0.029956   
4_payment_confirmation_page  0.009550  0.007152  0.007679  0.007631  0.002311   

month                                         4            
gender                           Male    Female      Male  
page                                                       
1_home_page                  1.000000  1.000000  1.000000  
2_search_page                0.389868  0.400375  0.403144  
3_payment_page               0.025903  0.025419  0.027397  
4_payment_confirmation_page  0.001586  0.001784  0.002283

## Выводы:

1. Мобилки конвертят лучше десктопов (или конвертили, пока кто-то не сломал страницу);
1. Большая часть аудитории сидит с десктопа, знать бы сколько приносит (и сколько он стоит) мобильный юзер и десктопный, можно было бы прикинуть стоит ли больше топить в одну из платформ;
1. По имеющимся данным не удается выявить отличия в поведении на сайте мужчин и женщин;
1. Оба пола одинаково представлены на сайте.

И еще:
1. pandas — очень крутой инструмент!
1. pivot_table открывает много дверей :)
1. без графиков отчет выглядит скучновато и читается трудно.